In [10]:
import argparse
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.optim.lr_scheduler import StepLR
import matplotlib.pyplot as plt
from foolbox import PyTorchModel, accuracy, samples
from foolbox.attacks import LinfPGD, FGSM
from trainers import Trainer, FGSMTrainer
from robustbench.model_zoo.models import Carmon2019UnlabeledNet
from utils import adversarial_accuracy, fgsm_
%load_ext autoreload
%autoreload 2
%aimport Nets, trainers

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
# setup
device = torch.device("cuda")
batch_size = 128
transform = transform = transforms.Compose(
            [transforms.ToTensor(),
             transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
)
        
normalized_min = (0 - 0.5) / 0.5
normalized_max = (1 - 0.5) / 0.5
train_dataset = datasets.CIFAR10(root='./data', train=True,
                                download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size,
                                  shuffle=True, num_workers=2)
test_dataset = datasets.CIFAR10(root='./data', train=False,
                               download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size,
                                 shuffle=False, num_workers=2)
classes = classes = ('plane', 'car', 'bird', 'cat',
   'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
preprocessing = dict(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5], axis=-3)

Files already downloaded and verified
Files already downloaded and verified


In [4]:
model = Carmon2019UnlabeledNet().to(device).eval()
model.load_state_dict(torch.load("models/cifar_10_wide_res_net_22e.model"))

<All keys matched successfully>

In [38]:
fmodel = PyTorchModel(model, bounds=(0, 1), preprocessing=preprocessing)
images, labels = samples(fmodel, dataset="cifar10", batchsize=16)
labels = labels.type(torch.cuda.LongTensor)
clean_acc = accuracy(fmodel, images, labels)
clean_acc

0.9375

In [66]:
# apply the attack
attack = LinfPGD()
epsilons = [
    0.0,
    0.0002,
    0.0005,
    0.0008,
    0.001,
    0.0015,
    0.002,
    0.003,
    0.01,
    0.1,
    0.3,
    0.5,
    1.0,
]
raw_advs, clipped_advs, success = attack(fmodel, images, labels, epsilons=epsilons)

# # calculate and report the robust accuracy (the accuracy of the model when
# # it is attacked)
robust_accuracy = 1 - success.type(torch.cuda.FloatTensor).mean(axis=-1)
print("robust accuracy for perturbations with")
for eps, acc in zip(epsilons, robust_accuracy):
    print(f"  Linf norm ≤ {eps:<6}: {acc.item() * 100:4.1f} %")

robust accuracy for perturbations with
  Linf norm ≤ 0.0   :  0.0 %
